In [14]:
import pip
from warnings import filterwarnings
filterwarnings("ignore")
def import_or_install(package:str):
    try:
        __import__(package)
        return 'Found'
    except ImportError:
        pip.main(['install', package]) 
        return 'Installed'
import os
print(os.environ['CONDA_DEFAULT_ENV'])
packages = ['pandas', 'tqdm', 'numpy','psutil', 'tensorflow', 'tables', 'ipywidgets']
[import_or_install(i) for i in packages]

In [27]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.backend as K
from tqdm import tqdm
import datetime
from pathlib import Path
import os
import time
import pandas as pd
import numpy as np
import psutil
from IPython.display import display, HTML, clear_output
from functions import *
import ipywidgets as widgets
# Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set_style('darkgrid')

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
# %matplotlib inline

In [28]:
# Set Paths
news_db = "news_db.h5"
DATA_PATH = 'models_DB.h5'
model_V1 = "./CharModelDev/CharModel-v1/daily.h5"
model_V2 = './CharModelDev/CharModel-v2/daily.h5'

In [29]:
# with pd.HDFStore(news_db) as store:
#     keys = list(store.keys())
#     news = pd.DataFrame()
#     for i in tqdm(keys): 
#         news = news.append(store[i])  

# news = news.reset_index().set_index('versionCreated')
# news.index = news.index.strftime('%Y-%m-%d')
# news.index = pd.to_datetime(news.index)
# news = news.drop(['index','storyId', 'sourceCode'], axis = 1)
# news.index.name = 'date'
# news.info()

In [30]:
#Set Processor to Run computations in
print(tf.config.list_physical_devices(device_type=None))
print(tf.config.experimental.list_logical_devices())
tf.config.optimizer.set_jit(True)
gpus = tf.config.list_physical_devices('XLA_CPU')
if gpus:
  # Restrict TensorFlow to only use some XLA_CPU
    try:
        tf.config.experimental.set_visible_devices(gpus[:], 'XLA_CPU')
        logical_gpus = tf.config.experimental.list_logical_devices('XLA_CPU')
        print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Visible devices must be set at program startup
        print(e)

# fix random seed for reproducibility
K.clear_session()
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)


#Load the models and accesories
with tf.device(tf.config.experimental.list_logical_devices()[-1]):
    daily_V1 = tf.keras.models.load_model(model_V1, compile=False)
    HeNormal = tf.keras.initializers.he_normal()
    daily_V2 = tf.keras.models.load_model(model_V2, custom_objects={'HeNormal': HeNormal},compile=False)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[LogicalDevice(name='/device:CPU:0', device_type='CPU')]


In [31]:
# # headline = "Amazon's Sales Growth Costs a Fortune in Shipping and Fulfillment" + " Jeff Bezos, Bill Gates and other tech luminaries react to Biden's victory" + " Amazon rolls out rewards program that makes it easier for drivers to get work" + " TECH Alibaba cloud growth outpaces Amazon and Microsoft as Chinese tech giant pushes for profitability"
# # headline = "<s>ZOETIS INC <ZTS.N>: CREDIT SUISSE RAISES PRICE TARGET TO $192 FROM $182 ZOETIS INC <ZTS.N>: BOFA GLOBAL RESEARCH RAISES PRICE OBJECTIVE TO $175 FROM $170 NYSE ORDER IMBALANCE <ZTS.N> 77562.0 SHARES ON SELL SIDE<\s>"
# headline = 'High earnings next quarter'
# # headline = "Joe Biden" 
# # sample = "Donald Trump" 
# sample = '<s>' + headline + '<\s' 
# print(sample)
# sample = encode2bytes(sample)
# print(sample)
# # sample = tf.ragged.constant(sample)
# sample = tf.squeeze(sample, )
# sample = tf.expand_dims(sample, 0).numpy()
# print(sample)
# sample.shape

In [32]:
# predict = daily_V1(sample).numpy()[0][0]
# print("Probability from Headlines: %f" % predict)

In [33]:
# headline = '<s>' + headline + '<\s' 
# sample_2 = headline.encode('utf-8')

# tokenizer = tokenize() 
# sample_2 = tokenizer.texts_to_sequences([sample_2])

# predict_v2 = daily_V2.predict(sample_2)[0,0]
# print("Probability from Headlines: %f" % predict_v2)

In [34]:
# trained_embeddings = daily_V2.get_layer('EmbedLayer').get_weights()[0]

In [35]:
# trained_embeddings.shape

In [36]:
news_headline_input = widgets.Textarea(
    value='',
    placeholder="Pass in news headline for the respective ticker here please!",
    description='New Headlines:',
    width = 500,
    disabled=False,
    layout={'width': 'max-content'}
)


model_1 = widgets.Button(
    description='Press to Get Model 1 Prediction',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check',# (FontAwesome names without the `fa-` prefix)
    layout={'width': 'max-content'}, # If the items' names are long
)

model_2 = widgets.Button(
    description='Press to Get Model 2 Prediction',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check',# (FontAwesome names without the `fa-` prefix)
    layout={'width': 'max-content'}, # If the items' names are long
)


def model1_preds(_):
    if news_headline_input.value == "":
        result = ui_display(message="Please pass in news headlines for a specific ticker in the box above to get a prediction")
        model_1_box.children = [model_1, result] 
    else:
        headline = news_headline_input.value
        headline = '<s>' + headline + '<\s' 
        # print(sample)
        sample = encode2bytes(headline)
        # print(sample)
        # sample = tf.ragged.constant(sample)
        sample = tf.squeeze(sample, )
        sample = tf.expand_dims(sample, 0).numpy()
        # print(sample)
        predict = daily_V1(sample).numpy()[0][0]
        app_type = ui_display(title = "Model 1 Predictions (127 Bits Embeddings)")
        result = ui_display(subtitle = "Probability from Headlines: %f" % predict, message='This is the probability to buy at market close today and sell market close tomorrow')        
        model_1_box.children = [model_1, app_type, result] 

def model2_preds(_):
    if news_headline_input.value == "":
        result = ui_display(message="Please pass in news headlines for a specific ticker in the box above to get a prediction")
        model_2_box.children = [model_2, result] 
    else:        
        headline = news_headline_input.value
        headline = '<s>' + headline + '<\s'
        sample_2 = headline.encode('utf-8')

        tokenizer = tokenize() 
        sample_2 = tokenizer.texts_to_sequences([sample_2])

        predict_v2 = daily_V2.predict(sample_2)[0,0]
        app_type = ui_display(title = "Model 2 Predictions (256 Bits Embeddings)")
        result = ui_display(subtitle = "Probability from Headlines: %f" % predict_v2, message='This is the probability to buy at market close today and sell market close tomorrow')        
        model_2_box.children = [model_2, app_type, result] 
        
def ui_display(df = None, 
               title = None,
               subtitle = None,
               message = None,
               clear = False):
    out = widgets.Output()
    with out:
        if title != None:
            display(HTML("<h1> %s </h1>"%title))
        if subtitle != None:
             display(HTML("<h2> %s </h2>"%subtitle))
        if message != None:
            # clear_output()
            display(message)    
            # display(message)
        if isinstance(df, pd.DataFrame):
            display(df)
    return out

In [37]:
title = ui_display(title= "Event Driven Trading (Byte Level Language Modelling) - UCBerkeley Demo")
subtle = ui_display(message= "News headlines from OverNight concatenated for next day Buy/Sell Probability/Signal")


status_label = widgets.Label()
status_label.layout.width = '1000px'

model_1_box = widgets.VBox(children=[model_1, status_label]) #widgets.VBox([sql_run,widgets.HBox([denygrp, type_formula], box_style = "success")])
model_2_box = widgets.VBox(children=[model_2, status_label])



intiate_box = widgets.VBox([title, subtle, widgets.HBox([news_headline_input], box_style = "success"), widgets.HBox([model_1_box, model_2_box])])
model_1.on_click(model1_preds)
model_2.on_click(model2_preds)


# button_run2.on_click(button_run_on_click2)
# display(widgets.HBox([main_box, main_box2]))
intiate_box